# 第9章 了解深度學習處理時間序列資料的原理(1-5節)
接下來要學習RNN與CNN處理時間序列資料的原理

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                #Colaboratory環境的設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter9

In [ ]:
#設定函式庫
!pip install -q -r ./requirements.txt

## 9-2 利用RNN預測正弦波


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import os

#固定亂數種子值
seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
%matplotlib inline

#載入資料
data = np.loadtxt("./rnn_sin_40_80.csv")
plt.plot(data[:500])
plt.show()

In [ ]:
#將history_steps步驟數用於輸入，再加工資料，以便預測future_steps步驟數
def create_dataset(data, history_steps, future_steps):
    input_data = []
    output_data= []
    
    for i in range(len(data)-history_steps-future_steps):
        input_data.append([[val] for val in data[i:i+history_steps]])
        output_data.append(data[i+history_steps:i+history_steps+future_steps])
    
    return np.array(input_data), np.array(output_data)

train_data = data[:int(len(data) * 0.75)]
test_data = data[int(len(data) * 0.75):]

#建立可根據10個步驟的資料量預測5步未來的資料
history_steps = 10
future_steps = 5
x_train, y_train = create_dataset(train_data, history_steps, future_steps)
x_test, y_test   = create_dataset(test_data, history_steps, future_steps)  

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam

#建立模型
model_rnn = Sequential()  
model_rnn.add(SimpleRNN(units=future_steps, input_shape=(history_steps,1),return_sequences=False))
model_rnn.add(Dense(future_steps,activation="linear"))  
model_rnn.compile(optimizer = Adam(lr=0.001), loss="mean_squared_error",)

#顯示模型的構造
print(model_rnn.summary())

#開始學習
history = model_rnn.fit(x_train, y_train, batch_size=32, epochs=500, verbose=1)

## 9-3 試著評估預測以RNN預測正弦波的結果

In [ ]:
#利用學習完成的模型預測
y_pred = model_rnn.predict(x_test)

plt.figure(figsize=(15,5))
#藍色為預測值，橘色為實際值
plt.plot([p[0] for p in y_pred],color="blue",label="pred")
plt.plot([p[0] for p in y_test],color="orange",label="actual")
plt.legend()
plt.show()

In [ ]:
#參數為學習完成的模型，用於預測的值，實際值
def predict_all(model, x_test, y_test):

    #只使用第一個history_steps。
    x_tmp = x_test[0]
    pred_result = []
    for index in range(len(y_test)):
        #利用x_tmp的資料預測
        pred = model.predict(np.array([x_tmp]))

        #刪除x_tmp的第一筆資料、再將預測所得的第一張資料存入這個陣列的尾部
        x_tmp = np.append(x_tmp[1:,], pred[0][0].reshape(1,1), axis=0)
        pred_result.append(pred[0][0])
        
    plt.figure(figsize=(15,5))
    #顯示預測值與實際值
    plt.plot(pred_result,color="blue", label="pred")
    plt.plot([p[0] for p in y_test],color="orange", label="actual")
    plt.legend()
    plt.show()

#開始預測
predict_all(model_rnn, x_test, y_test)

## 9-4 試著利用CNN預測正弦波


In [ ]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D, GlobalMaxPooling1D

#建立模型
model_conv = Sequential()
model_conv.add(Conv1D(filters=64, kernel_size=4, strides=1, padding='same', input_shape=x_train.shape[1:], activation='relu'))
model_conv.add(Conv1D(filters=128, kernel_size=4,strides=1, padding='same', activation='relu'))
model_conv.add(GlobalMaxPooling1D())
model_conv.add(Dense(future_steps, activation='tanh'))
model_conv.compile(optimizer = Adam(lr=0.001), loss='mean_squared_error')

#顯示模型的構造
print(model_conv.summary())

#開始預測
history = model_conv.fit(x_train, y_train, batch_size=32, epochs=500, verbose=1)

In [ ]:
#開始預測
y_pred = model_conv.predict(x_test)

plt.figure(figsize=(15,5))
plt.plot([p[0] for p in y_pred],color="blue",label="pred")
plt.plot([p[0] for p in y_test],color="orange",label="actual")
plt.legend()
plt.show()

In [ ]:
predict_all(model_conv, x_test, y_test)

## 9-5 提升預測正弦波的精準度


In [ ]:
history_steps_v2 = 40
future_steps_v2 = 10

#根據40步的資料預測接下來10步的資料
x_train_v2, y_train_v2 = create_dataset(train_data, history_steps_v2, future_steps_v2)
x_test_v2, y_test_v2   = create_dataset(test_data, history_steps_v2, future_steps_v2)  


print(x_train_v2.shape)
print(y_train_v2.shape)

In [ ]:
#建立RNN的模型
model_rnn_v2 = Sequential()  
model_rnn_v2.add(SimpleRNN(units=future_steps_v2, input_shape=(history_steps_v2,1),return_sequences=False))
model_rnn_v2.add(Dense(future_steps_v2,activation="linear"))  
model_rnn_v2.compile(optimizer = Adam(lr=0.001), loss="mean_squared_error",)

#顯示模型的構造
print(model_rnn_v2.summary())
        
#開始學習
history = model_rnn_v2.fit(x_train_v2, y_train_v2, batch_size=32, epochs=500, verbose=1)

In [ ]:
predict_all(model_rnn_v2, x_test_v2, y_test_v2)

In [ ]:
#建立CNN的模型
model_conv_v2 = Sequential()
model_conv_v2.add(Conv1D(filters=64, kernel_size=4, strides=1, padding='same', input_shape=x_train_v2.shape[1:], activation='relu'))
model_conv_v2.add(Conv1D(filters=128, kernel_size=4,strides=1, padding='same', activation='relu'))
model_conv_v2.add(GlobalMaxPooling1D())
model_conv_v2.add(Dense(future_steps_v2, activation='tanh'))
model_conv_v2.compile(optimizer = Adam(lr=0.001), loss='mean_squared_error')

#顯示模型的構造
print(model_conv_v2.summary())

#開始學習
history = model_conv_v2.fit(x_train_v2, y_train_v2, batch_size=32, epochs=500, verbose=1)

In [ ]:
predict_all(model_conv_v2, x_test_v2, y_test_v2)